In [301]:
import pandas as pd
import numpy as np
import urllib.request, json 
import xmltodict
from collections import Counter
import pickle
import time

In [293]:
mapper = pd.read_csv("Data/PMC-ids.csv")
mapper = mapper[pd.notnull(mapper['PMID'])]
mapper.head()
mapper.shape

/Users/jonghalee/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,5,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(4320673, 12)

In [294]:
discovery = pd.read_csv("Data/DiscoveryEngine/DiscoveryEngine_Dataset_2_28_2018.csv")
discovery.shape

(2716, 8)

In [295]:
discovery = discovery.merge(mapper, how = "left", left_on = "manuscript_DOI", right_on = "DOI")
discovery.shape

(2716, 20)

In [296]:
havePMID = discovery.loc[~np.isnan(discovery['PMID'])]
havePMID['PMID'] = havePMID['PMID'].astype(int)
havePMID.shape

/Users/jonghalee/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(1432, 20)

In [304]:
baseurl = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id="
retmode = "&retmode=xml&api_key=4253acd8482343f9a87e9563b1a14c873908"

In [305]:
# VERSION 1
allYears = havePMID['Year'].astype(int)
BuzzDict = {}
MeshDict = {}
idDict = {}
np.random.seed(999)
for y in allYears:
    yearData = mapper.loc[(mapper["Year"] == y) & (~mapper["PMID"].isin(havePMID['PMID']))]
    
    ids = list(yearData['PMID'].unique().astype(int))
    ids = [str(i) for i in ids]
    ids = np.random.choice(ids, size = 1000, replace = False)
    
    allMesh = []
    allKey = []
    track = 1
    
    for i in ids:
        track += 1
        
        # Print line and save temporary changes:
        if track % 200 == 0:
            print("Starting PMID " + i + "....")
            BuzzDict[y] = Counter(allKey)
            idDict[y] = ids
            with open('BuzzwordData.pkl', 'wb') as output:
                pickle.dump(BuzzDict, output, pickle.HIGHEST_PROTOCOL)
                pickle.dump(MeshDict, output, pickle.HIGHEST_PROTOCOL)
                pickle.dump(idDict, output, pickle.HIGHEST_PROTOCOL)

        # Open URL, read data
        file = urllib.request.urlopen(baseurl + i + retmode)
        data = file.read()
        file.close()

        # Parse XML data
        data = xmltodict.parse(data)
        
        #Meshwords
        try:
            meshData = data['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['MeshHeadingList']['MeshHeading']
            for di in meshData:
                word = di['DescriptorName']['#text']
                words = word.split(" ")
                allMesh.extend(words)     
        except KeyError:
            pass

        #Keywords
        try:
            keyData = data['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['KeywordList']['Keyword']
            for k in keyData:
                word = k['#text']
                words = word.split(" ")
                allKey.extend(words)        
        except KeyError:
            continue
        time.sleep(0.1 + np.random.uniform(0,0.1))

    BuzzDict[y] = Counter(allKey)
    MeshDict[y] = Counter(allMesh)
    idDict[y] = ids

URLError: <urlopen error [Errno 60] Operation timed out>

In [290]:
with open('BuzzwordData.pkl', 'wb') as output:
    pickle.dump(BuzzDict, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(MeshDict, output, pickle.HIGHEST_PROTOCOL)
    pickle.dump(idDict, output, pickle.HIGHEST_PROTOCOL)

In [237]:
# VERSION 2 

# allYears = havePMID['Year'].astype(int)
# allDf = pd.DataFrame()
# #y = 2007
# for y in allYears:
#     yearData = mapper.loc[(mapper["Year"] == y) & (~mapper["PMID"].isin(havePMID['PMID']))]
    
#     ids = list(yearData['PMID'].unique())
#     ids = [str(i) for i in ids]
    
#     yrVec = [y] * len(ids)
#     WordDf = pd.DataFrame({'PMID': ids, 'Year': yrVec})
    
#     allMesh = []
#     allKey = []
#     track = 1
#     for i in ids[:100]:
#         # Print line:
#         #if track % 10 == 0:
#         print("Starting PMID " + i + "....")

#         # Open URL, read data
#         file = urllib.request.urlopen(baseurl + i + retmode)
#         data = file.read()
#         file.close()

#         # Parse XML data
#         data = xmltodict.parse(data)

#         #Meshwords
#         meshWords = []
#         try:
#             meshData = data['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['MeshHeadingList']['MeshHeading']
#             for di in meshData:
#                 word = di['DescriptorName']['#text']
#                 words = word.split(" ")
#                 meshWords.extend(words)     
#         except KeyError:
#             pass
#         allMesh.append(meshWords)

#         #Keywords
#         keyWords = []
#         try:
#             keyData = data['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['KeywordList']['Keyword']
#             for k in keyData:
#                 word = k['#text']
#                 words = word.split(" ")
#                 keyWords.extend(words)        
#         except KeyError:
#             pass
#         allKey.append(keyWords)
#         track += 1
        
#     WordDf['Meshwords'] = allMesh
#     WordDf['Keywords'] = allKey
#     allDf = pd.concat([allDf, WordDf], axis = 1)

Starting PMID 17193884.0....
Starting PMID 17195339.0....
Starting PMID 17193877.0....
Starting PMID 17193876.0....
Starting PMID 17193878.0....
Starting PMID 17193883.0....
Starting PMID 17193879.0....
Starting PMID 17193881.0....
Starting PMID 17193885.0....
Starting PMID 17193880.0....
Starting PMID 17193882.0....
Starting PMID 17193875.0....
Starting PMID 17111375.0....
Starting PMID 17159004.0....
Starting PMID 17105992.0....
Starting PMID 17095540.0....
Starting PMID 17119099.0....
Starting PMID 17095543.0....
Starting PMID 17119100.0....
Starting PMID 17123956.0....
Starting PMID 17119098.0....
Starting PMID 17105991.0....
Starting PMID 17095542.0....
Starting PMID 17135484.0....
Starting PMID 17105994.0....
Starting PMID 17095544.0....
Starting PMID 17123955.0....
Starting PMID 17095541.0....
Starting PMID 17105995.0....
Starting PMID 17105993.0....
Starting PMID 17199409.0....
Starting PMID 17200697.0....
Starting PMID 17200695.0....
Starting PMID 17200696.0....
Starting PMID 

KeyboardInterrupt: 

In [ ]:
# JSON PARSING TEST
# with urllib.request.urlopen(baseurl + "17913979" + retmode) as url:
#     data = json.loads(url.read().decode())
#     test = pd.read_json(data['result'], orient = "index")


# MANUAL STRING PARSING TEST
# testURL = urllib.request.urlopen(baseurl + "17913979" + retmode)
# impLines = []
# for line in testURL:
#     if ("b'OT" in str(line)) or ("b'MH" in str(line)):
#         impLines.append(line)
# impLines
# hi = [''.join(e for e in str(string) if e.isalnum()) for string in impLines]
# hi